## Import Packages

In [20]:
import argparse
import math

import numpy as np
from skimage.io import imread, imsave
from PIL import Image

## (a) Color Quantization

In [21]:
def median_cut_quantize(img_arr):
    # when it reaches the end, color quantize
    r_average = round(np.mean(img_arr[:, 0]))
    g_average = round(np.mean(img_arr[:, 1]))
    b_average = round(np.mean(img_arr[:, 2]))

    LUT.append([r_average, g_average, b_average])
    MAP.append(img_arr)
  

def split_into_buckets(img_arr, depth):
    if len(img_arr) == 0:
      return

    if depth == 0:
      median_cut_quantize(img_arr)
      return

    if depth%3 == 0: #R
      sort_arr = img_arr[img_arr[:, 0].argsort()]
    elif depth%3 == 2: #G
      sort_arr = img_arr[img_arr[:, 1].argsort()]
    else: #B
      sort_arr = img_arr[img_arr[:, 2].argsort()]
      
    median_index = int((len(sort_arr) + 1) / 2)

    # split the array into two blocks
    split_into_buckets(sort_arr[0:median_index], depth - 1)
    split_into_buckets(sort_arr[median_index:], depth - 1)


In [22]:
output_path = "out/median_cut3.png"
input_path = "img/Lenna.jpg"
n_bit_colors = 3

In [23]:
img = imread(input_path)

flattened_img_array = []
for rindex, rows in enumerate(img):
    for cindex, color in enumerate(rows):
        flattened_img_array.append([color[0], color[1], color[2], rindex, cindex])

flattened_img_array = np.array(flattened_img_array)

LUT = []
MAP = []

# start the splitting process
split_into_buckets(flattened_img_array, n_bit_colors)


LUT_3_bit = np.array(LUT)
print('Colors in the Look-Up-Table:')
print(LUT_3_bit)

quantized_arr = np.zeros_like(img)
for x, y in zip(LUT_3_bit, MAP):
  for i in y:
    quantized_arr[i[3]][i[4]] = x

quantized_img = Image.fromarray(quantized_arr, 'RGB')
quantized_img.save(output_path)
quantized_img.show()

unique_colors = np.unique(quantized_arr.reshape(-1, quantized_arr.shape[2]), axis=0)
print('Check the number of colors in the result image:')
print(len(unique_colors))


Colors in the Look-Up-Table:
[[100  24  62]
 [119  42  79]
 [175  71  82]
 [169 101 120]
 [211  99  94]
 [215 120 115]
 [225 149 125]
 [229 187 168]]
Check the number of colors in the result image:
8


In [24]:
#compute MSE
mse = np.mean((img - quantized_img)**2)
print(mse)

70.18580088661005


### n=6


In [25]:
output_path = "out/median_cut6.png"
input_path = "img/Lenna.jpg"
n_bit_colors = 6

In [26]:
img = imread(input_path)

flattened_img_array = []
for rindex, rows in enumerate(img):
    for cindex, color in enumerate(rows):
        flattened_img_array.append([color[0], color[1], color[2], rindex, cindex])

flattened_img_array = np.array(flattened_img_array)

LUT = []
MAP = []

# start the splitting process
split_into_buckets(flattened_img_array, n_bit_colors)

LUT_6_bit = np.array(LUT)
print('Colors in the Look-Up-Table:')
print(LUT_6_bit)

quantized_arr = np.zeros_like(img)
for x, y in zip(LUT_6_bit, MAP):
  for i in y:
    quantized_arr[i[3]][i[4]] = x

quantized_img = Image.fromarray(quantized_arr, 'RGB')
quantized_img.save(output_path)
quantized_img.show()

unique_colors = np.unique(quantized_arr.reshape(-1, quantized_arr.shape[2]), axis=0)
print('Check the number of colors in the result image:')
print(len(unique_colors))

Colors in the Look-Up-Table:
[[ 84  13  56]
 [ 84  16  63]
 [ 90  22  61]
 [ 89  23  66]
 [104  23  59]
 [101  25  65]
 [124  35  63]
 [125  39  67]
 [ 92  25  70]
 [ 92  26  76]
 [102  36  75]
 [105  42  92]
 [138  45  70]
 [127  46  83]
 [157  56  73]
 [136  56  90]
 [168  63  75]
 [160  63  81]
 [170  69  80]
 [163  73  87]
 [182  68  76]
 [182  71  83]
 [189  80  83]
 [189  84  90]
 [149  74 101]
 [133  75 125]
 [166 102 116]
 [153 113 156]
 [189  90  97]
 [187 103 114]
 [190 121 117]
 [190 132 135]
 [202  85  84]
 [203  89  93]
 [205  97  92]
 [205 100  99]
 [215  94  90]
 [216  98  98]
 [221 112  95]
 [222 115 101]
 [208 106 107]
 [207 114 118]
 [205 129 118]
 [205 130 127]
 [222 112 106]
 [223 116 115]
 [226 129 107]
 [225 129 119]
 [213 136 120]
 [210 137 130]
 [215 145 126]
 [217 150 137]
 [234 141 110]
 [233 145 124]
 [237 159 121]
 [239 177 134]
 [215 159 150]
 [215 169 162]
 [221 185 169]
 [223 194 184]
 [236 182 154]
 [235 192 177]
 [245 207 160]
 [238 209 189]]
Check the 

In [27]:
#compute MSE
mse = np.mean((img - quantized_img)**2)
print(mse)

28.25556801794584


## (b) Error Diffusion Dithering

### n=3

In [28]:
output_path = "out/error_diffusion_dithering_3.png"
input_path = "img/Lenna.jpg"
n_bit_colors = 3

In [29]:
def get_new_val(old_val, nc):
    """
    Get the "closest" color to old_val 
    """
    if nc == 3:
      diff = np.abs(LUT_3_bit - old_val)
      idx = np.argmin((np.sum(diff ** 2, axis=1)) ** 0.5)
      new_val = LUT_3_bit[idx]
    else: #nc=6
      diff = np.abs(LUT_6_bit - old_val)
      idx = np.argmin((np.sum(diff ** 2, axis=1)) ** 0.5)
      new_val = LUT_6_bit[idx]

    return new_val

def fs_dither(arr, nc):
    arr = np.array(img, dtype='f8')
    width = img.shape[1]
    height = img.shape[0]

    for ir in range(height):
      for ic in range(width):
        old_val = arr[ir, ic].copy()
        new_val = get_new_val(old_val, nc)
        arr[ir, ic] = new_val
        err = old_val - new_val
          
        if ic < width - 1:
          arr[ir, ic+1] += err * 7/16
          arr[ir, ic+1] = np.clip(arr[ir, ic+1], 0, 255)
        if ir < height - 1:
          if ic > 0:
            arr[ir+1, ic-1] += err * 3/16
            arr[ir+1, ic-1] = np.clip(arr[ir+1, ic-1], 0, 255)
          arr[ir+1, ic] += err * 5/16
          arr[ir+1, ic] = np.clip(arr[ir+1, ic], 0, 255)
          if ic < width - 1:
            arr[ir+1, ic+1] += err / 16
            arr[ir+1, ic+1] = np.clip(arr[ir+1, ic+1], 0, 255)
    carr = np.array(arr, dtype=np.uint8)
    return carr

In [30]:
img = imread(input_path)

dithering_arr = fs_dither(img, n_bit_colors)
dithering_img = Image.fromarray(dithering_arr, 'RGB')
dithering_img.save(output_path)
dithering_img.show()

unique_colors = np.unique(dithering_arr.reshape(-1, quantized_arr.shape[2]), axis=0)
print('Check the number of colors in the result image:')
print(len(unique_colors))

Check the number of colors in the result image:
8


In [16]:
#compute MSE
mse = np.mean((img - dithering_img)**2)
print(mse)

73.52571035624632


### n=6

In [17]:
output_path = "out/error_diffusion_dithering_6.png"
input_path = "img/Lenna.jpg"
n_bit_colors = 6

In [18]:
img = imread(input_path)

dithering_arr = fs_dither(img, n_bit_colors)
dithering_img = Image.fromarray(dithering_arr, 'RGB')
dithering_img.save(output_path)
dithering_img.show()

unique_colors = np.unique(dithering_arr.reshape(-1, quantized_arr.shape[2]), axis=0)
print('Check the number of colors in the result image:')
print(len(unique_colors))

Check the number of colors in the result image:
64


In [19]:
#compute MSE
mse = np.mean((img - dithering_img)**2)
print(mse)

37.47621575068098
